# WEB SCRAPING

First we need to import all the necessary packages.

In [2]:
import requests
import lxml.html as lh
import pandas as pd
import numpy as np

Then go to the url and right click on the element to scrape.Click on inspect and find out the tag containing the dataset. 

In [3]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
#Create a handle, page, to handle the contents of the website
page = requests.get(url)
#Store the contents of the website under doc
doc = lh.fromstring(page.content)
#Parse data that are stored between <tr>..</tr> of HTML
tr_elements = doc.xpath('//tr')

In [4]:
#Check the length of the first 12 rows
[len(T) for T in tr_elements[:12]]

[3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]

Now we need to get the first row of the table that is the header row and print it.



In [5]:
tr_elements = doc.xpath('//tr')
#Create empty list
col=[]
i=0
#For each row, store each first element (header) and an empty list
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    print ('%d:"%s"'%(i,name))
    col.append((name,[]))

1:"Postal Code
"
2:"Borough
"
3:"Neighborhood
"


Append the data from the columns row after row.

In [6]:
#Since out first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        col[i][1].append(data)
        #Increment i for the next column
        i+=1

We can check if the data from all the columns have been successfully obtained by checking their length.

In [7]:
[len(C) for (title,C) in col]

[181, 181, 181]

Now we are sure of the length of the column.So we can convert the data into a dataframe using Pandas.

In [8]:
Dict={title:column for (title,column) in col}
df=pd.DataFrame(Dict)
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M1A\n,Not assigned\n,Not assigned\n
1,M2A\n,Not assigned\n,Not assigned\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [9]:
print("The number of rows in the dataframe are {}".format(df.shape[0]))

The number of rows in the dataframe are 181


Replace Not assigned in the borough column to NaN so that they can be easily removed.

In [10]:
df.replace('Not assigned\n',np.nan,inplace=True)
df.dropna(subset=['Borough\n'],axis=0,inplace=True)
df.reset_index(drop=True,inplace=True)

In [11]:
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M3A\n,North York\n,Parkwoods\n
1,M4A\n,North York\n,Victoria Village\n
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights\n"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government\n"


In [12]:
print("The number of rows in the dataframe are {}".format(df.shape[0]))

The number of rows in the dataframe are 104


After removing the rows where Borough is unavailable ,there are now 104 rows instead of 181.

But my data needs to be cleaned up a bit removing all whitespace characters.

In [13]:
df.replace(r'\s', '', regex = True, inplace = True)

In [14]:
df.head()

,Postal Code\n,Borough\n,Neighborhood\n
0,M3A,NorthYork,Parkwoods
1,M4A,NorthYork,VictoriaVillage
2,M5A,DowntownToronto,"RegentPark,Harbourfront"
3,M6A,NorthYork,"LawrenceManor,LawrenceHeights"
4,M7A,DowntownToronto,"Queen'sPark,OntarioProvincialGovernment"


Cleaning the row headers:

In [15]:
df.columns=['Postal Code','Borough','Neighborhood']
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,NorthYork,Parkwoods
1,M4A,NorthYork,VictoriaVillage
2,M5A,DowntownToronto,"RegentPark,Harbourfront"
3,M6A,NorthYork,"LawrenceManor,LawrenceHeights"
4,M7A,DowntownToronto,"Queen'sPark,OntarioProvincialGovernment"


Group the neighborhoods in the same borough.

In [16]:
# group neighborhoods in the same borough
df_grouped = df.groupby(["Postal Code", "Borough"], as_index=False).agg(lambda x: ", ".join(x))
df_grouped.head()

,Postal Code,Borough,Neighborhood
0,,Canadianpostalcodes,
1,M1B,Scarborough,"Malvern,Rouge"
2,M1C,Scarborough,"RougeHill,PortUnion,HighlandCreek"
3,M1E,Scarborough,"Guildwood,Morningside,WestHill"
4,M1G,Scarborough,Woburn


In [22]:
df_grouped=df_grouped.iloc[1:,:]
df_grouped.reset_index(drop=True,inplace=True)

In [23]:
df_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1C,Scarborough,"RougeHill,PortUnion,HighlandCreek"
1,M1E,Scarborough,"Guildwood,Morningside,WestHill"
2,M1G,Scarborough,Woburn
3,M1H,Scarborough,Cedarbrae
4,M1J,Scarborough,ScarboroughVillage


In [24]:
# for Neighborhood=NaN, make the value the same as Borough
for index, row in df_grouped.iterrows():
    if row["Neighborhood"] == np.nan:
        row["Neighborhood"] = row["Borough"]
        
df_grouped.head()

,Postal Code,Borough,Neighborhood
0,M1C,Scarborough,"RougeHill,PortUnion,HighlandCreek"
1,M1E,Scarborough,"Guildwood,Morningside,WestHill"
2,M1G,Scarborough,Woburn
3,M1H,Scarborough,Cedarbrae
4,M1J,Scarborough,ScarboroughVillage


In [25]:
print("The number of rows in the dataframe are {}".format(df.shape[0]))

The number of rows in the dataframe are 103
